In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from scipy.stats import poisson
from scipy.stats import nbinom
import sys
from pathlib import Path
import time

## TRANSMISSION MODEL

In [ ]:
#########################################################################################

#compute median and CI

def add_median_CI(DF):
    df = DF.copy()
    df['p1'] = df[[i for i in range(n_runs)]].quantile(0.025, axis=1)
    df['median'] = df[[i for i in range(n_runs)]].median(axis=1)
    df['p2'] = df[[i for i in range(n_runs)]].quantile(0.975, axis=1)
    return df

#########################################################################################

# functions to read and modify the matrix

# reshape the array in a 4x4 matrix
def keep(df):
    C = []
    for i in range(ages*ages):
        c = df[0][i]
        C.append(c)
    C = np.array(C)
    C = np.reshape(C, (ages,ages))
    return C

#construct the four contact matrices
def extract_matrix(matrix, x_test_a,x_test_s):
    m_p = keep(matrix) # healthy behavior (matrix for the individuals in the prodromic phase)
    m_as = x_test_a*keep(matrix)*0.1 + (1-x_test_a)*keep(matrix)      # asymptomatic
    m_t = x_test_s*keep(matrix)*0.1 + (1-x_test_s)*keep(matrix)       # pauci-symptomatic and mild
    m_ss = x_test_s*keep(matrix)*0.1 + (1-x_test_s)*keep(matrix)*0.25  # severe symptoms

    return m_p,m_as,m_t,m_ss

#########################################################################################

def seir(run, u, parms, t, 
         scaling,                   
         dm_p, dm_as, dm_t, dm_s, 
         LD_scenario): 
    
    """
    One time step (day) spreading of the compartmental, age-stratified model.
    INPUT
    run: index of the current run
    u: dict containing the number of individuals (incidence and prevalence) for each compartment in the previous timestep
    parms: list of epidemiological parameters
    t: current timestep
    scaling: dictionary of scaling factors
    dm_p, dm_as, dm_t, dm_s: dictionaries of contact matrices
    LD_scenario: string defining scenario for the projection ('curfew' or 'strict_LD' or 'moderate_LD')
    RETURNS: updated version of dict u
    """

    #quantities of interest
    S = u['S']

    new_E = u['Y_E']
    E = u['E']
    new_I_p = u['Y_I_p']
    I_p = u['I_p']
    new_I_as = u['Y_I_as']
    I_as = u['I_as']
    new_I_ps = u['Y_I_ps']
    I_ps = u['I_ps']
    new_I_ms = u['Y_I_ms']
    I_ms = u['I_ms']
    new_I_ss = u['Y_I_ss']
    I_ss = u['I_ss']
    new_H = u['Y_H']
    H = u['H']
    new_R = u['Y_R']
    R = u['R']
    
    N_tot = u['N_tot']

    bet, sigm, thet, gamm, N, dt = parms
    
    # week 6 curfew w/ schools open
    if t < start_winter_holidays:
        C_h = dm_p['curfew']*scaling['holidays']
        C_as = dm_as['curfew']*scaling['holidays']
        C_t = dm_t['curfew']*scaling['holidays']
        C_s = dm_s['curfew']*scaling['holidays']
        
    # week 7-8 curfew w/ school holidays
    elif t>= start_winter_holidays and t < end_winter_holidays:
        C_h = dm_p['holidays']*scaling['holidays']
        C_as = dm_as['holidays']*scaling['holidays']
        C_t = dm_t['holidays']*scaling['holidays']
        C_s = dm_s['holidays']*scaling['holidays']  
        
    # week 9-11 curfew w/ schools open
    elif t >= end_winter_holidays and t < LD_start:
        C_h = dm_p['curfew']*scaling['curfew']
        C_as = dm_as['curfew']*scaling['curfew']
        C_t = dm_t['curfew']*scaling['curfew']
        C_s = dm_s['curfew']*scaling['curfew']        
    
    elif t>= LD_start and t<LD_end:
        if LD_typ == 'no-change':
            sys.exit()
        else:
            if LD_typ == 'strict_LD':
                dm_p_SC = dm_p[LD_typ]
                dm_as_SC = dm_as[LD_typ]
                dm_t_SC = dm_t[LD_typ]
                dm_s_SC = dm_s[LD_typ]
            elif LD_typ == 'moderate_LD':
                dm_p_SC = dm_p['moderate_LD_SC']
                dm_as_SC = dm_as['moderate_LD_SC']
                dm_t_SC = dm_t['moderate_LD_SC']
                dm_s_SC = dm_s['moderate_LD_SC']
            else:
                sys.exit()
                
            if loss_adherence == 'no_loss':
                
                if t >= start_spring_holidays and t < end_spring_holidays:            
                    C_h = dm_p_SC*scaling[LD_typ]
                    C_as = dm_as_SC*scaling[LD_typ]
                    C_t = dm_t_SC*scaling[LD_typ]
                    C_s = dm_s_SC*scaling[LD_typ]
                else:
                    C_h = dm_p[LD_typ]*scaling[LD_typ]
                    C_as = dm_as[LD_typ]*scaling[LD_typ]
                    C_t = dm_t[LD_typ]*scaling[LD_typ]
                    C_s = dm_s[LD_typ]*scaling[LD_typ]   
                    
            elif loss_adherence == 'limited_loss':
                
                if t >= start_spring_holidays and t < end_spring_holidays:            
                    if t < LD_start + 7*step_adherence:
                        C_h = dm_p_SC*scaling[LD_typ]
                        C_as = dm_as_SC*scaling[LD_typ]
                        C_t = dm_t_SC*scaling[LD_typ]
                        C_s = dm_s_SC*scaling[LD_typ]
                    else:
                        C_h = dm_p_SC*scaling[LD_typ]*rel_increase
                        C_as = dm_as_SC*scaling[LD_typ]*rel_increase
                        C_t = dm_t_SC*scaling[LD_typ]*rel_increase
                        C_s = dm_s_SC*scaling[LD_typ]*rel_increase                          
                else:
                    if t < LD_start + 7*step_adherence:
                        C_h = dm_p[LD_typ]*scaling[LD_typ]
                        C_as = dm_as[LD_typ]*scaling[LD_typ]
                        C_t = dm_t[LD_typ]*scaling[LD_typ]
                        C_s = dm_s[LD_typ]*scaling[LD_typ] 
                    else:
                        C_h = dm_p[LD_typ]*scaling[LD_typ]*rel_increase 
                        C_as = dm_as[LD_typ]*scaling[LD_typ]*rel_increase
                        C_t = dm_t[LD_typ]*scaling[LD_typ]*rel_increase
                        C_s = dm_s[LD_typ]*scaling[LD_typ]*rel_increase
                        
            elif loss_adherence == 'continuous_loss':
                
                if t >= start_spring_holidays and t < end_spring_holidays:          
                    if t < LD_start + 7*step_adherence:
                        C_h = dm_p_SC*scaling[LD_typ]
                        C_as = dm_as_SC*scaling[LD_typ]
                        C_t = dm_t_SC*scaling[LD_typ]
                        C_s = dm_s_SC*scaling[LD_typ]
                    elif t < LD_start + 7*step_adherence*2:
                        C_h = dm_p_SC*scaling[LD_typ]*rel_increase
                        C_as = dm_as_SC*scaling[LD_typ]*rel_increase
                        C_t = dm_t_SC*scaling[LD_typ]*rel_increase
                        C_s = dm_s_SC*scaling[LD_typ]*rel_increase 
                    elif t < LD_start + 7*step_adherence*3:
                        C_h = dm_p_SC*scaling[LD_typ]*rel_increase*rel_increase
                        C_as = dm_as_SC*scaling[LD_typ]*rel_increase*rel_increase
                        C_t = dm_t_SC*scaling[LD_typ]*rel_increase*rel_increase
                        C_s = dm_s_SC*scaling[LD_typ]*rel_increase*rel_increase 
                    elif t < LD_start + 7*step_adherence*4:
                        C_h = dm_p_SC*scaling[LD_typ]*rel_increase*rel_increase*rel_increase 
                        C_as = dm_as_SC*scaling[LD_typ]*rel_increase*rel_increase*rel_increase 
                        C_t = dm_t_SC*scaling[LD_typ]*rel_increase*rel_increase*rel_increase 
                        C_s = dm_s_SC*scaling[LD_typ]*rel_increase*rel_increase*rel_increase 
                    else:
                        sys.exit()
                else:
                    if t < LD_start + 7*step_adherence:
                        C_h = dm_p[LD_typ]*scaling[LD_typ]
                        C_as = dm_as[LD_typ]*scaling[LD_typ]
                        C_t = dm_t[LD_typ]*scaling[LD_typ]
                        C_s = dm_s[LD_typ]*scaling[LD_typ] 
                    elif t < LD_start + 7*step_adherence*2:
                        C_h = dm_p[LD_typ]*scaling[LD_typ]*rel_increase 
                        C_as = dm_as[LD_typ]*scaling[LD_typ]*rel_increase
                        C_t = dm_t[LD_typ]*scaling[LD_typ]*rel_increase
                        C_s = dm_s[LD_typ]*scaling[LD_typ]*rel_increase 
                    elif t < LD_start + 7*step_adherence*3:
                        C_h = dm_p[LD_typ]*scaling[LD_typ]*rel_increase*rel_increase
                        C_as = dm_as[LD_typ]*scaling[LD_typ]*rel_increase*rel_increase
                        C_t = dm_t[LD_typ]*scaling[LD_typ]*rel_increase*rel_increase
                        C_s = dm_s[LD_typ]*scaling[LD_typ]*rel_increase*rel_increase 
                    elif t < LD_start + 7*step_adherence*4:
                        C_h = dm_p[LD_typ]*scaling[LD_typ]*rel_increase*rel_increase*rel_increase
                        C_as = dm_as[LD_typ]*scaling[LD_typ]*rel_increase*rel_increase*rel_increase
                        C_t = dm_t[LD_typ]*scaling[LD_typ]*rel_increase*rel_increase*rel_increase
                        C_s = dm_s[LD_typ]*scaling[LD_typ]*rel_increase*rel_increase*rel_increase 
                    else:
                        sys.exit()
            else:
                sys.exit()
                
    elif t>=LD_end:
        if t >= start_spring_holidays and t < end_spring_holidays:
            C_h = dm_p['holidays']*scaling['holidays']
            C_as = dm_as['holidays']*scaling['holidays']
            C_t = dm_t['holidays']*scaling['holidays']
            C_s = dm_s['holidays']*scaling['holidays']     
        else:
            C_h = dm_p['curfew']*scaling['curfew']
            C_as = dm_as['curfew']*scaling['curfew']
            C_t = dm_t['curfew']*scaling['curfew']
            C_s = dm_s['curfew']*scaling['curfew']

    else:
        sys.exit()

    lambd = {'wild': [0]*ages, 'VOC':[0]*ages, 'wild_vax': [0]*ages, 'VOC_vax':[0]*ages}
    
    # transmissibility advantage wrt wild strain non vaccinated
    eta = {'wild': 1.0, 'VOC': trans_increase, 
           'wild_vax': 1.0*(1.0-alpha_trans['wild']), 'VOC_vax': trans_increase*(1.0-alpha_trans['VOC'])}
    
    # compute force of infection
    for strain in ['wild','VOC','wild_vax','VOC_vax']:
        for age in range(ages):
            l = 0
            for age2 in range(ages):
                l += eta[strain]*susc[age]*r[age2]*bet*C_h[age,age2]*I_p[strain][age2]/N
                l += eta[strain]*susc[age]*r[age2]*bet*C_as[age,age2]*I_as[strain][age2]/N
                l += eta[strain]*susc[age]*r[age2]*bet*C_t[age,age2]*I_ps[strain][age2]/N
                l += eta[strain]*susc[age]*bet*C_t[age,age2]*I_ms[strain][age2]/N
                l += eta[strain]*susc[age]*bet*C_s[age,age2]*I_ss[strain][age2]/N
            lambd[strain][age] = l
        lambd[strain] = np.array(lambd[strain])
    
    # compute transition probabilities for entering in each comparment
    in_E = {'wild': 1.0 - np.exp(-(lambd['wild']+lambd['wild_vax'])*dt), 
            'VOC': 1.0 - np.exp(-(lambd['VOC']+lambd['VOC_vax'])*dt),
            'wild_vax': 1.0 - np.exp(-(lambd['wild']+lambd['wild_vax'])*dt*(1.0-alpha_infect['wild'])),
            'VOC_vax': 1.0 - np.exp(-(lambd['VOC']+lambd['VOC_vax'])*dt*(1.0-alpha_infect['VOC']))}
    
    in_I_p = {'wild': np.array([sigm*dt]*ages), 'VOC': np.array([sigm*dt]*ages),
             'wild_vax': np.array([sigm*dt]*ages), 'VOC_vax': np.array([sigm*dt]*ages)}
    
    in_I = {'wild': np.array([thet*dt]*ages), 'VOC': np.array([thet*dt]*ages),
           'wild_vax': np.array([thet*dt]*ages), 'VOC_vax': np.array([thet*dt]*ages)}
    
    in_R = {'wild': np.array([gamm*dt]*ages), 'VOC': np.array([gamm*dt]*ages),
           'wild_vax': np.array([gamm*dt]*ages), 'VOC_vax': np.array([gamm*dt]*ages)}
    
    in_H = {'wild': np.array([gamm*dt]*ages), 'VOC': np.array([gamm*dt]*ages),
           'wild_vax': np.array([gamm*dt]*ages), 'VOC_vax': np.array([gamm*dt]*ages)}
    
    # transition events (sampled by binomial or multinomial distributions with corresponding transition probabilities)
    
    # infection events
    for age in range(ages):
        #non vaccinated
        force_of_infection = np.array([in_E['wild'][age],in_E['VOC'][age],
                                       1.0-in_E['wild'][age]-in_E['VOC'][age]])
        
        new_E['wild'][age],new_E['VOC'][age],res = np.random.multinomial(S['no_vax'][age],force_of_infection)
        
        #vaccinated
        force_of_infection = np.array([in_E['wild_vax'][age],in_E['VOC_vax'][age],
                                       1.0-in_E['wild_vax'][age]-in_E['VOC_vax'][age]])
        new_E['wild_vax'][age],new_E['VOC_vax'][age],res = np.random.multinomial(S['vax'][age],force_of_infection)
    
    # spontaneous transitions
    recovery_as = {}
    recovery_ps = {}
    recovery_ms = {}
    recovery_h = {}
    
    list_strains=['wild','VOC','wild_vax','VOC_vax']
        
    for strain in list_strains:
        
        recovery_as[strain] = [0]*ages
        recovery_ps[strain] = [0]*ages
        recovery_ms[strain] = [0]*ages
        recovery_h[strain] = [0]*ages
        
        for age in range(ages):
            new_I_p[strain][age] = np.random.binomial(E[strain][age],in_I_p[strain][age])

            trans_I = np.array([p_as_v[strain][age]*in_I[strain][age], 
                                p_ps_v[strain][age]*in_I[strain][age], 
                                p_ms_v[strain][age]*in_I[strain][age], 
                                p_ss_v[strain][age]*in_I[strain][age], 
                                1.-in_I[strain][age]])
            new_I_as[strain][age], new_I_ps[strain][age], new_I_ms[strain][age], new_I_ss[strain][age], res = np.random.multinomial(I_p[strain][age],trans_I)

            recovery_as[strain][age] = np.random.binomial(I_as[strain][age],in_R[strain][age])
            recovery_ps[strain][age] = np.random.binomial(I_ps[strain][age],in_R[strain][age])
            recovery_ms[strain][age] = np.random.binomial(I_ms[strain][age],in_R[strain][age])

            new_H[strain][age] = np.random.binomial(I_ss[strain][age],in_H[strain][age])
            recovery_h[strain][age] = np.random.binomial(H[strain][age], in_R[strain][age])

            new_R[strain][age] = recovery_as[strain][age] + recovery_ps[strain][age] + recovery_ms[strain][age] + recovery_h[strain][age]

    #update compartments
    
    for age in range(ages):  
        
        S['no_vax'][age] = S['no_vax'][age] - new_E['wild'][age] - new_E['VOC'][age]
        S['vax'][age] = S['vax'][age] - new_E['wild_vax'][age] - new_E['VOC_vax'][age] 
        
        for strain in list_strains:      
            E[strain][age] = E[strain][age] + new_E[strain][age] - new_I_p[strain][age]
            I_p[strain][age] = I_p[strain][age] + new_I_p[strain][age] - new_I_as[strain][age] - new_I_ps[strain][age] - new_I_ms[strain][age] - new_I_ss[strain][age]
            I_as[strain][age] = I_as[strain][age] + new_I_as[strain][age] - recovery_as[strain][age]
            I_ps[strain][age] = I_ps[strain][age] + new_I_ps[strain][age] - recovery_ps[strain][age]
            I_ms[strain][age] = I_ms[strain][age] + new_I_ms[strain][age] - recovery_ms[strain][age]
            I_ss[strain][age] = I_ss[strain][age] + new_I_ss[strain][age] - new_H[strain][age]
            H[strain][age] = H[strain][age] + new_H[strain][age] - recovery_h[strain][age]
            R[strain][age] = R[strain][age] + new_R[strain][age]
        
        wild_compts = E['wild'][age]+I_p['wild'][age]+I_as['wild'][age]+I_ps['wild'][age]+I_ms['wild'][age]+I_ss['wild'][age]+H['wild'][age]+R['wild'][age]
        
        VOC_compts = E['VOC'][age]+I_p['VOC'][age]+I_as['VOC'][age]+I_ps['VOC'][age]+I_ms['VOC'][age]+I_ss['VOC'][age]+H['VOC'][age]+R['VOC'][age]
        
        wild_compts_vax = E['wild_vax'][age]+I_p['wild_vax'][age]+I_as['wild_vax'][age]+I_ps['wild_vax'][age]+I_ms['wild_vax'][age]+I_ss['wild_vax'][age]+H['wild_vax'][age]+R['wild_vax'][age]
        
        VOC_compts_vax = E['VOC_vax'][age]+I_p['VOC_vax'][age]+I_as['VOC_vax'][age]+I_ps['VOC_vax'][age]+I_ms['VOC_vax'][age]+I_ss['VOC_vax'][age]+H['VOC_vax'][age]+R['VOC_vax'][age]

        N_tot[age] = S['no_vax'][age] + S['vax'][age] + wild_compts + VOC_compts + wild_compts_vax + VOC_compts_vax
    
    # vaccination campaign
    if t >= vacc_phase_1 and t < vacc_end:
        vaccinated_per_day = {}
        if t >= vacc_phase_1 and t < vacc_phase_2:
            vaccinated_per_day['sen'] = int((N_s/N_s_france)*doses_per_day['phase_1']/2.)
            vaccinated_per_day['adu'] = int((N_a/N_a_france)*doses_per_day['phase_1']/2.)
        elif t >= vacc_phase_2 and t < vacc_phase_3:
            vaccinated_per_day['sen'] = int((N_s/N_s_france)*doses_per_day['phase_2']/2.)
            vaccinated_per_day['adu'] = int((N_a/N_a_france)*doses_per_day['phase_2']/2.)
        elif t >= vacc_phase_3 and t < vacc_end:
            vaccinated_per_day['sen'] = int((N_s/N_s_france)*doses_per_day['phase_3']/2.)        
            vaccinated_per_day['adu'] = int((N_a/N_a_france)*doses_per_day['phase_3']/2.)        
        else:
            sys.exit()

        vaccinabili_adu = S['no_vax'][2]+R['wild'][2]+R['VOC'][2]
        vaccinabili_sen = S['no_vax'][3]+R['wild'][3]+R['VOC'][3]
        
        perc_vacc_adu = 1.0-vaccinabili_adu/N_a
        perc_vacc_sen = 1.0-vaccinabili_sen/N_s
        
        if perc_vacc_sen < (1.0 - hesitancy[3]):
 
            if vaccinabili_sen > 0:
                trans_vax = np.array([S['no_vax'][3]/float(vaccinabili_sen), 
                                    R['wild'][3]/float(vaccinabili_sen), 
                                    R['VOC'][3]/float(vaccinabili_sen),0])
            else: 
                trans_vax = np.array([0,0,0,1.])

            new_S_vax, new_R_wild_vax, new_R_VOC_vax, res = np.random.multinomial(vaccinated_per_day['sen'], trans_vax)

            vax = min(new_R_wild_vax,R['wild'][3])
            R['wild'][3] = R['wild'][3]-vax
            R['wild_vax'][3] = R['wild_vax'][3]+vax

            vax = min(new_R_VOC_vax,R['VOC'][3])
            R['VOC'][3] = R['VOC'][3]-vax
            R['VOC_vax'][3] = R['VOC_vax'][3]+vax

            vax = min(S['no_vax'][3],new_S_vax)
            S['no_vax'][3] = S['no_vax'][3]-vax
            S['vax'][3] = S['vax'][3]+vax

        elif perc_vacc_adu < (1.0 - hesitancy[2]):
            
            if vaccinabili_adu > 0:
                trans_vax = np.array([S['no_vax'][2]/float(vaccinabili_adu), 
                                    R['wild'][2]/float(vaccinabili_adu), 
                                    R['VOC'][2]/float(vaccinabili_adu),0])
            else: 
                trans_vax = np.array([0,0,0,1.])

            new_S_vax, new_R_wild_vax, new_R_VOC_vax, res = np.random.multinomial(vaccinated_per_day['adu'], trans_vax)

            vax = min(new_R_wild_vax,R['wild'][2])
            R['wild'][2] = R['wild'][2]-vax
            R['wild_vax'][2] = R['wild_vax'][2]+vax

            vax = min(new_R_VOC_vax,R['VOC'][2])
            R['VOC'][2] = R['VOC'][2]-vax
            R['VOC_vax'][2] = R['VOC_vax'][2]+vax

            vax = min(S['no_vax'][2],new_S_vax)
            S['no_vax'][2] = S['no_vax'][2]-vax
            S['vax'][2] = S['vax'][2]+vax

    return  {'t':t, 'S': S, 'E':E, 'I_p':I_p, 
             'I_as':I_as,'I_ps':I_ps,'I_ms':I_ms,'I_ss':I_ss, 'H':H, 'R':R,
             'Y_E':new_E,'Y_I_p':new_I_p,'Y_I_as':new_I_as,
             'Y_I_ps':new_I_ps,'Y_I_ms':new_I_ms,'Y_I_ss':new_I_ss,
             'Y_H':new_H, 'Y_R':new_R,'N_tot':N_tot}

def simulate(run,
             scaling,                   #dictionary of scaling factors
             dm_p, dm_as, dm_t, dm_s,   #dictionary with contact matrices 
             LD_scenario): #string indicating type of lockdown
    
    """
    Simulation of one single stochastic run of the compartmental, age-stratified model.
    INPUT
    run: index of the current run
    scaling: dictionary of scaling factors
    dm_p, dm_as, dm_t, dm_s: dictionaries of contact matrices
    LD_scenario: string defining scenario for the projection ('curfew' or 'strict_LD' or 'moderate_LD')
    RETURNS: dict containing the number of individuals for each compartment, for each timestep, for one run
    """
    
    parms = [beta, sigma, theta, gamma, N, delta_t]
    
    tf = t_stop
    
    t = np.arange(tf)
    
    S = {'no_vax' : np.zeros((tf,ages)), 'vax' : np.zeros((tf,ages))} 
    E = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
        'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    I_p = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
          'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    I_as = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    I_ps = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    I_ms = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    I_ss = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    H = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    R = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    Y_E = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    Y_I_p = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    Y_I_as = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    Y_I_ps = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    Y_I_ms = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    Y_I_ss = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    Y_H = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    Y_R = {'wild' : np.zeros((tf,ages)), 'VOC' : np.zeros((tf,ages)),
           'wild_vax' : np.zeros((tf,ages)), 'VOC_vax' : np.zeros((tf,ages))} 
    
    N_tot = np.zeros((tf,ages))
        
    result = {'t':t, 'S': S, 
              'E':E, 'I_p':I_p, 
              'I_as':I_as,'I_ps':I_ps,'I_ms':I_ms,'I_ss':I_ss, 'H':H,'R':R,
              'Y_E':Y_E,'Y_I_p':Y_I_p,
              'Y_I_as':Y_I_as,'Y_I_ps':Y_I_ps,'Y_I_ms':Y_I_ms,'Y_I_ss':Y_I_ss,
              'Y_H':Y_H,'Y_R':Y_R,'N_tot':N_tot}
        
    #initial condition
    u = {'t':0,
         'S': {'no_vax': [0]*ages, 'vax': [0]*ages}, 
         'E':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'I_p':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'I_as':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'I_ps':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'I_ms':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'I_ss':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'H':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'R':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'Y_E':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'Y_I_p':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'Y_I_as':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'Y_I_ps':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'Y_I_ms':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'Y_I_ss':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'Y_H':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'Y_R':{'wild':[0]*ages,'VOC':[0]*ages,'wild_vax':[0]*ages,'VOC_vax':[0]*ages},
         'N_tot':[N_c,N_t,N_a,N_s]}
    
    # initialize with prevalence in each compartment as of Feb 7, 2021 (end of week 5)
    for age in range(ages):
        u['S']['no_vax'][age] = configuration['S_no_vax_{}'.format(age)].iloc[run]
        u['S']['vax'][age] = configuration['S_vax_{}'.format(age)].iloc[run]

        for strain in ['wild','VOC','wild_vax','VOC_vax']:
            u['E'][strain][age] = int(configuration['E_{}_{}'.format(strain,age)].iloc[run])
            u['I_p'][strain][age] = int(configuration['I_p_{}_{}'.format(strain,age)].iloc[run])
            u['I_as'][strain][age] = int(configuration['I_as_{}_{}'.format(strain,age)].iloc[run])
            u['I_ps'][strain][age] = int(configuration['I_ps_{}_{}'.format(strain,age)].iloc[run])
            u['I_ms'][strain][age] = int(configuration['I_ms_{}_{}'.format(strain,age)].iloc[run])
            u['I_ss'][strain][age] = int(configuration['I_ss_{}_{}'.format(strain,age)].iloc[run])
            u['H'][strain][age] = int(configuration['H_{}_{}'.format(strain,age)].iloc[run])
            u['R'][strain][age] = int(configuration['R_{}_{}'.format(strain,age)].iloc[run])

    for c in result.keys():
            #e.g. result['I_p'] is a dictionary with two keys, wild and VOC
            #result['I_p']['wild'] is an array with length = t_stop
            #each elements is a list with length = ages
        if c not in ['t','S','N_tot']:
            for strain in ['wild','VOC','wild_vax','VOC_vax']:
                result[c][strain][0] = u[c][strain]
        elif c =='S':
            for status in ['no_vax', 'vax']:
                result[c][status][0] = u[c][status]
        else:
            result[c][0] = u[c]
    
    for j in range(1,tf):
        #j is the time index
        u = seir(run, u, parms, t[j],
                 scaling,                   
                 dm_p, dm_as, dm_t, dm_s,   
                 LD_scenario) 
        
        for c in result.keys(): #list of compartments
            #I insert in position j the list [x,x,x,x] which indicates number of individuals 
            #in compartment c with strain at the timestep j
            if c not in ['t','S','N_tot']:
                for strain in ['wild','VOC','wild_vax','VOC_vax']:
                    result[c][strain][j] = u[c][strain]
            elif c =='S':
                for status in ['no_vax', 'vax']:
                    result[c][status][j] = u[c][status]
            else:
                result[c][j] = u[c]
                 
    return result

def run_simulation(LD_scenario):
    
    """
    LD_scenario: string defining scenario for the projection ('curfew' or 'strict_LD' or 'moderate_LD')
    """

    #read matrices
    
    #strict lockdown
    strict_LD = pd.read_table(initial_path+'/matrices/LD1.txt', header = None, sep=' ')

    #moderate lockdown
    path_matrix=initial_path+'/matrices/LD2_SO.txt'
    moderate_LD=pd.read_table(path_matrix, header = None, sep=' ')
    
    #moderate lockdown w/ school closed
    path_matrix=initial_path+'/matrices/LD2_SC.txt'
    moderate_LD_SC=pd.read_table(path_matrix, header = None, sep=' ')
    
    #curfew
    path_matrix=initial_path+'/matrices/curfew.txt'
    curfew=pd.read_table(path_matrix, header = None, sep=' ') 
    
    #school holidays
    path_matrix=initial_path+'/matrices/school_holidays.txt'
    holidays=pd.read_table(path_matrix, header = None, sep=' ')  
    
    dm_p = {}
    dm_as = {}
    dm_t = {}
    dm_s = {}
    
    name = 'curfew'
    dm_p[name], dm_as[name], dm_t[name], dm_s[name] = extract_matrix(curfew, x_test_a=0.5, x_test_s=0.5)
    name = 'strict_LD'
    dm_p[name], dm_as[name], dm_t[name], dm_s[name] = extract_matrix(strict_LD, x_test_a=0.5, x_test_s=0.5)
    name = 'moderate_LD'
    dm_p[name], dm_as[name], dm_t[name], dm_s[name] = extract_matrix(moderate_LD, x_test_a=0.5, x_test_s=0.5)
    name = 'moderate_LD_SC'
    dm_p[name], dm_as[name], dm_t[name], dm_s[name] = extract_matrix(moderate_LD_SC, x_test_a=0.5, x_test_s=0.5)
    name = 'holidays'
    dm_p[name], dm_as[name], dm_t[name], dm_s[name] = extract_matrix(holidays, x_test_a=0.5, x_test_s=0.5)

    #initialize dataframes to contain the results of each simulation
    
    out_H = {'wild': pd.DataFrame(np.arange(t_stop)), 'VOC': pd.DataFrame(np.arange(t_stop)),
            'wild_vax': pd.DataFrame(np.arange(t_stop)), 'VOC_vax': pd.DataFrame(np.arange(t_stop))}

    #for each run
    for n in range(n_runs):
        run = n #number of the current run
        out = simulate(run,
                       scaling,                   
                       dm_p, dm_as, dm_t, dm_s,   
                       LD_scenario)
        
        # extract quantities of interest, e.g. daily hospital admissions
        for strain in ['wild','VOC','wild_vax','VOC_vax']:
            # save the timeseries in the column of a dataframe  
            out_H[strain][n] = pd.DataFrame(out['Y_H'][strain]).sum(axis=1)

    return {'new_H':out_H}

## INPUT

In [ ]:
initial_path='./input'

### details of the simulations

In [ ]:
ages = 4  #number of age classes
n_runs = 250 # number of runs
delta_t = 1 #timestep (days), e.g. delta_t=1 means a timestep of 1 day

### epidemiological parameters

In [ ]:
incubation = 3.7       # incubation period (non infectious) (days)
prodromic = 1.5        # prodromic phase (I_p)
infection = 2.3        # infectious period (days)

# transition rates
sigma=1./incubation
theta=1./prodromic
gamma=1./infection    

increase_H = 1.64   # increase in hospitalization risk due to Alpha variant

# 'wild' = historical strains
# VOC = Alpha variant

asint = {'wild': 0.4, 'VOC': 0.4}
#probabily of asymptomatic infection
p_as = {'wild': np.array([asint['wild'],asint['wild'],asint['wild'],asint['wild']]),
        'VOC': np.array([asint['VOC'],asint['VOC'],asint['VOC'],asint['VOC']])}
#probability of symptomatic infection
ps = {'wild': np.array([1.,1.,0.2,0.2]),  
        'VOC': np.array([1.,1.,0.2,0.2])}

#probability of few symptoms
p_ps = {'wild': np.multiply(1.-p_as['wild'], ps['wild']),
        'VOC': np.multiply(1.-p_as['VOC'], ps['VOC'])}

ss_adults_input = 0.0296 
ss_seniors_input = 0.254

ss_adults = {'wild': ss_adults_input, 'VOC': ss_adults_input*increase_H}
ss_seniors = {'wild': ss_seniors_input, 'VOC': ss_seniors_input*increase_H}

ms = {'wild': np.array([0.,0.,0.8-ss_adults['wild'],0.8-ss_seniors['wild']]), 
      'VOC': np.array([0.,0.,0.8-ss_adults['VOC'],0.8-ss_seniors['VOC']])}
ss = {'wild': np.array([0.,0.,ss_adults['wild'],ss_seniors['wild']]), 
      'VOC': np.array([0.,0.,ss_adults['VOC'],ss_seniors['VOC']])}
#probability of mild symptoms
p_ms = {'wild': np.multiply(1.-p_as['wild'], ms['wild']),
        'VOC': np.multiply(1.-p_as['VOC'], ms['VOC'])}
#probability of severe symptoms
p_ss = {'wild': np.multiply(1.-p_as['wild'], ss['wild']),
        'VOC': np.multiply(1.-p_as['VOC'], ss['VOC'])}

r = np.array([0.25,0.55,0.55,0.55])  # relative infectiousness
susc = np.array([0.5,0.5,1,1])       # relative susceptibility

### population

In [ ]:
# population of IDF
N_c = 1590000   # young children
N_t = 1550000   # adolescents
N_a = 7250000   # adults
N_s = 1890000   # seniors

N = N_c+N_t+N_a+N_s

# population of France
N_a_france = 36060000
N_s_france = 13450000

### transmission rate

In [ ]:
beta=0.0895

trans_increase = 1.59  # increase in transmission due to Alpha variant

scaling = {'strict_LD': 1.23, 'moderate_LD': 1.61, 'curfew': 1.38, 'holidays': 1.44}

### loss of adherence

In [ ]:
rel_increase = 1.109158  # relative increase in the transmission rate

step_adherence = 2 #drop in adherence every 2 weeks

### set calendar

In [ ]:
# simulation starts on February 7, 2021
calendar = pd.DataFrame(pd.date_range(dt.date(2021, 2, 7), periods=365))

delay_LD = 4 # delay in effect of lockdown is 7 days (4 days + 3 days due to compartmental structure)
LD_start = calendar[calendar[0]=='2021-03-22'].index[0] + delay_LD

# winter school holidays
start_winter_holidays = calendar[calendar[0]=='2021-02-15'].index[0]
end_winter_holidays = calendar[calendar[0]=='2021-03-01'].index[0]
# spring school holidays
start_spring_holidays = calendar[calendar[0]=='2021-04-12'].index[0]
end_spring_holidays = calendar[calendar[0]=='2021-04-26'].index[0]

# final date of simulation
t_stop = calendar[calendar[0]=='2021-07-05'].index[0] 

### initial conditions

In [ ]:
# this file contains initial conditions for each compartment, for each runs
# initial condition for a run is sampled from a loggamma distribution
configuration = pd.read_csv(initial_path+'/configuration_loggamma_2021-02-07.csv')

### vaccination parameters

In [ ]:
# vaccine rhythm
doses_per_day = {'phase_1': 100000, 'phase_2': 400000, 'phase_3': 600000}

# vaccine hesitancy for each age class
# we use a maximum coverage of 50% for adults and 80% for seniors
# children and adolescents are not subject to vaccination
hesitancy = np.array([1.0,1.0,0.5,0.2])

# vaccine efficacy starts 3 weeks after injection
vax_delay = 3 #weeks

# phase 1
vacc_phase_1 = calendar[calendar[0]=='2021-02-08'].index[0]
# phase 2
vacc_phase_2 = calendar[calendar[0]=='2021-03-08'].index[0]
vacc_phase_2 = vacc_phase_2 + 7*vax_delay
# phase 3
vacc_phase_3 = calendar[calendar[0]=='2021-03-29'].index[0]
vacc_phase_3 = vacc_phase_3 + 7*vax_delay

vacc_end = t_stop

alpha_infect = {'wild': 0.75, 'VOC': 0.75}  # efficacy againts infection
alpha_trans = {'wild': 0.65, 'VOC': 0.65}   # efficacy againts transmission
alpha_sympt = {'wild': 0.8, 'VOC': 0.8}     # efficacy againts symptoms given infection, chosen so that efficacy againts symptom is 95%

# compute probability of developing symptoms according to strain and vaccination status
# asymptomatic
p_as_v = {'wild': p_as['wild'].copy(), 
          'VOC': p_as['VOC'].copy(),
'wild_vax': p_as['wild'] + alpha_sympt['wild']*(p_ms['wild']+p_ss['wild'])*(p_as['wild']/(p_as['wild']+p_ps['wild'])), 
'VOC_vax': p_as['VOC'] + alpha_sympt['VOC']*(p_ms['VOC']+p_ss['VOC'])*(p_as['VOC']/(p_as['VOC']+p_ps['VOC']))}
# pauci-symptomatic
p_ps_v = {'wild': p_ps['wild'].copy(), 
          'VOC': p_ps['VOC'].copy(),
'wild_vax': p_ps['wild'] + alpha_sympt['wild']*(p_ms['wild']+p_ss['wild'])*(p_ps['wild']/(p_as['wild']+p_ps['wild'])), 
'VOC_vax': p_ps['VOC'] + alpha_sympt['VOC']*(p_ms['VOC']+p_ss['VOC'])*(p_ps['VOC']/(p_as['VOC']+p_ps['VOC']))} 
# mild symptoms
p_ms_v = {'wild': p_ms['wild'].copy(), 
          'VOC': p_ms['VOC'].copy(),
          'wild_vax': p_ms['wild']*(1.0-alpha_sympt['wild']), 
          'VOC_vax': p_ms['VOC']*(1.0-alpha_sympt['VOC'])}
# severe symptoms
p_ss_v = {'wild': p_ss['wild'].copy(), 
          'VOC': p_ss['VOC'].copy(),
          'wild_vax': p_ss['wild']*(1.0-alpha_sympt['wild']), 
          'VOC_vax': p_ss['VOC']*(1.0-alpha_sympt['VOC'])}

## RUN SCENARIOS

In [ ]:
folder = './output'
Path(folder).mkdir(parents=True, exist_ok=True)

In [ ]:
LD_typ = 'curfew'

LD_end = LD_start

# run simulation

out = run_simulation(LD_scenario=LD_typ)

#######################################
comp_keys = ['new_H']

for c in range(len(comp_keys)):
    # sum compartments 
    ddf = out[comp_keys[c]]['VOC'].copy() + out[comp_keys[c]]['wild'].copy() + out[comp_keys[c]]['VOC_vax'].copy() + out[comp_keys[c]]['wild_vax'].copy()
    # add median and 95% CI
    ddf = add_median_CI(ddf)
    # add calendar
    ddf['time'] = calendar[0].iloc[:t_stop]
    # save output
    ddf.to_csv('{}/{}_{}.csv'.format(folder, comp_keys[c], LD_typ))

In [ ]:
LD_typ = 'strict_LD'

for loss_adherence in ['no_loss', 'continuous_loss']:
    
    for duration_LD in [2,4,6,8]:
        
        LD_end = LD_start+7*duration_LD

        # run simulation

        out = run_simulation(LD_scenario=LD_typ)

        #######################################
        comp_keys = ['new_H']

        for c in range(len(comp_keys)):
            # sum compartments 
            ddf = out[comp_keys[c]]['VOC'].copy() + out[comp_keys[c]]['wild'].copy() + out[comp_keys[c]]['VOC_vax'].copy() + out[comp_keys[c]]['wild_vax'].copy()
            # add median and 95% CI
            ddf = add_median_CI(ddf)
            # add calendar
            ddf['time'] = calendar[0].iloc[:t_stop]
            # save output
            ddf.to_csv('{}/{}_{}_{}_{}weeks.csv'.format(folder, comp_keys[c], LD_typ, loss_adherence, duration_LD))

In [ ]:
LD_typ = 'moderate_LD'

for loss_adherence in ['no_loss', 'limited_loss', 'continuous_loss']:
    
    for duration_LD in [2,4,6,8]:
        
        LD_end = LD_start+7*duration_LD

        # run simulation

        out = run_simulation(LD_scenario=LD_typ)

        #######################################
        comp_keys = ['new_H']

        for c in range(len(comp_keys)):
            # sum compartments 
            ddf = out[comp_keys[c]]['VOC'].copy() + out[comp_keys[c]]['wild'].copy() + out[comp_keys[c]]['VOC_vax'].copy() + out[comp_keys[c]]['wild_vax'].copy()
            # add median and 95% CI
            ddf = add_median_CI(ddf)
            # add calendar
            ddf['time'] = calendar[0].iloc[:t_stop]
            # save output
            ddf.to_csv('{}/{}_{}_{}_{}weeks.csv'.format(folder, comp_keys[c], LD_typ, loss_adherence, duration_LD))
            